# Arithmetic Operations And Aggregations


Import the LArray library:


In [ ]:
from larray import *

## Arithmetic operations


In [ ]:
arr = ndtest((3, 3))
arr

One can do all usual arithmetic operations on an array, it will apply the operation to all elements individually


In [ ]:
# addition
arr + 10

In [ ]:
# multiplication
arr * 2

In [ ]:
# 'true' division
arr / 2

In [ ]:
# 'floor' division
arr // 2

<div class="alert alert-warning">
**Warning:** Python has two different division operators: 

- the 'true' division (/) always returns a float.
- the 'floor' division (//) returns an integer result (discarding any fractional result).
</div>

In [ ]:
# % means modulo (aka remainder of division)
arr % 5

In [ ]:
# ** means raising to the power
arr ** 3

More interestingly, binary operators as above also works between two arrays:


In [ ]:
# load the 'demography_eurostat' dataset
demography_eurostat = load_example_data('demography_eurostat')

# extract the 'population' array
population = demography_eurostat.population
population

In [ ]:
aggregation_matrix = Array([[1, 0, 0], [0, 1, 1]], axes=(Axis('country=Belgium,France+Germany'), population.country))
aggregation_matrix

In [ ]:
# @ means matrix product
aggregation_matrix @ population['Male']

<div class="alert alert-info">
**Note:** Be careful when mixing different data types.
You can use the method [astype](../_generated/larray.Array.astype.rst#larray.Array.astype) to change the data type of an array.
</div>


In [ ]:
aggregation_matrix = Array([[1, 0, 0], [0, 0.5, 0.5]], axes=(Axis('country=Belgium,France+Germany/2'), population.country))
aggregation_matrix

In [ ]:
aggregation_matrix @ population['Male']

In [ ]:
# force the resulting matrix to be an integer matrix
(aggregation_matrix @ population['Male']).astype(int)

### Axis order does not matter much (except for output)

You can do operations between arrays having different axes order.
The axis order of the result is the same as the left array


In [ ]:
# extract the 'births' array
births = demography_eurostat.births

# let's change the order of axes of the 'births' array
births_transposed = births.transpose()
births_transposed

In [ ]:
# LArray doesn't care of axes order when performing 
# arithmetic operations between arrays
population + births_transposed

### Axes must be compatible

Arithmetic operations between two arrays only works when they have compatible axes (i.e. same labels).

In [ ]:
# the 'population' and 'births' have compatible axes
population + births

In [ ]:
# Now, let's replace the country names by the country codes
births_codes = births.set_labels('country', ['BE', 'FR', 'DE'])
births_codes

In [ ]:
# arithmetic operations between arrays 
# having incompatible axes raise an error
try:
    population + births_codes
except Exception as e:
    print(type(e).__name__, e)

<div class="alert alert-warning">
    **Warning:** Operations between two arrays only works when they have compatible axes (i.e. same labels) but this behavior can be override via the [ignore_labels](../_generated/larray.Array.ignore_labels.rst#larray.Array.ignore_labels) method.
In that case only the position on the axis is used and not the labels.
Using this method is done at your own risk.
</div>


In [ ]:
# use the .ignore_labels() method on axis 'country'
# to avoid the incompatible axes error (risky)
population + births_codes.ignore_labels('country')

### Extra Or Missing Axes (Broadcasting)

The condition that axes must be compatible only applies on common axes. 
Arithmetic operations between two arrays can be performed even if the second array has extra or missing axes compared to the first one:

In [ ]:
# let's define a 'multiplicator' vector with 
# one value defined for each gender
multiplicator = Array([-1, 1], axes=population.gender)
multiplicator

In [ ]:
# the multiplication below has been propagated to the 
# 'country' and 'time' axes.
# This behavior is called broadcasting
population * multiplicator

### Boolean Operations

Python comparison operators are: 

| Operator  | Meaning                 |
|-----------|-------------------------|
|``==``     | equal                   | 
|``!=``     | not equal               | 
|``>``      | greater than            | 
|``>=``     | greater than or equal   | 
|``<``      | less than               | 
|``<=``     | less than or equal      |

Applying a comparison operator on an array returns a boolean array:

In [ ]:
# test which values are greater than 10 millions
population > 10e6

Comparison operations can be combined using Python bitwise operators:

| Operator | Meaning                              |
|----------|------------------------------------- |
|  &       | and                                  |
|  \|      | or                                   |
|  ~       | not                                  |

In [ ]:
# test which values are greater than 10 millions and less than 40 millions
(population > 10e6) & (population < 40e6)

In [ ]:
# test which values are less than 10 millions or greater than 40 millions
(population < 10e6) | (population > 40e6)

In [ ]:
# test which values are not less than 10 millions
~(population < 10e6)

The returned boolean array can then be used in selections and assignments:

In [ ]:
population_copy = population.copy()

# set all values greater than 40 millions to 40 millions
population_copy[population_copy > 40e6] = 40e6
population_copy

Boolean operations can be made between arrays:

In [ ]:
# test where the two arrays have the same values
population == population_copy

To test if all values between are equals, use the [equals](../_generated/larray.Array.equals.rst#larray.Array.equals) method:

In [ ]:
population.equals(population_copy)

## Aggregates

The LArray library provides many aggregation functions. The list is given in the [Aggregation Functions](../api.rst#aggregation-functions) subsection of the [API Reference](../api.rst) page.

Aggregation operations can be performed on axes or groups. Axes and groups can be mixed. 

The main rules are: 

-  Axes are separated by commas ``,``
-  Groups belonging to the same axis are grouped inside parentheses ()


Calculate the sum along an axis:

In [ ]:
population.sum('gender')

or several axes (axes are separated by commas ``,``):

In [ ]:
population.sum('country', 'gender')

Calculate the sum along all axes except one by appending `_by` to the aggregation function:


In [ ]:
population.sum_by('time')

Calculate the sum along groups (the groups belonging to the same axis must grouped inside parentheses ()):


In [ ]:
even_years = population.time[2014::2] >> 'even_years'
odd_years = population.time[2013::2] >> 'odd_years'

population.sum((odd_years, even_years))

Mixing axes and groups in aggregations:


In [ ]:
population.sum('gender', (odd_years, even_years))